In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


connecting the kaggle for collecting necessary *dataset*

In [2]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


downloading the dataset with API key

In [3]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
  0% 0.00/15.1M [00:00<?, ?B/s]
100% 15.1M/15.1M [00:00<00:00, 1.15GB/s]


In [4]:
!unzip -q brain-mri-images-for-brain-tumor-detection.zip -d "/content/drive/MyDrive/brain_tumor_dataset"

splitting and saving them in different folders like Train and valid

In [5]:
import os, shutil
import random

source_dir = "/content/drive/MyDrive/brain_tumor_dataset"
target_dir = "/content/drive/MyDrive/brain_tumor_dataset"
train_split = 0.8

for split in ['train', 'val']:
    for category in ['yes', 'no']:
        os.makedirs(os.path.join(target_dir, split, category), exist_ok=True)

for category in ['yes', 'no']:
    files = os.listdir(os.path.join(source_dir, category))
    random.shuffle(files)
    split_idx = int(len(files) * train_split)
    train_files = files[:split_idx]
    val_files = files[split_idx:]

    for f in train_files:
        shutil.copy(
            os.path.join(source_dir, category, f),
            os.path.join(target_dir, 'train', category, f)
        )

    for f in val_files:
        shutil.copy(
            os.path.join(source_dir, category, f),
            os.path.join(target_dir, 'val', category, f)
        )

print(" Dataset split complete!")


 Dataset split complete!


In [7]:
!pip install tensorflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00


In [1]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

In [4]:
train_path = "/content/drive/MyDrive/brain_tumor_dataset/train"
val_path = "/content/drive/MyDrive/brain_tumor_dataset/val"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_path,
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

NUM_CLASSES = len(train_ds.class_names)

Found 202 files belonging to 2 classes.
Found 51 files belonging to 2 classes.


In [5]:
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

inputs = layers.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,562 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,052,133 (15.46 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "/content/drive/MyDrive/best_brain_tumor_model.h5"

checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    verbose=1,
    mode='min'
)


In [7]:
EPOCHS = 10

history = model.fit(
    train_ds.prefetch(tf.data.AUTOTUNE),
    validation_data=val_ds.prefetch(tf.data.AUTOTUNE),
    epochs=EPOCHS,
    callbacks=[checkpoint_cb]
)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3894 - loss: 0.8918
Epoch 1: val_loss improved from inf to 0.70191, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.3970 - loss: 0.8854 - val_accuracy: 0.5882 - val_loss: 0.7019
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6592 - loss: 0.6549
Epoch 2: val_loss improved from 0.70191 to 0.56540, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.6579 - loss: 0.6559 - val_accuracy: 0.7059 - val_loss: 0.5654
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6615 - loss: 0.6010
Epoch 3: val_loss improved from 0.56540 to 0.50867, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.6649 - loss: 0.5988 - val_accuracy: 0.7843 - val_loss: 0.5087
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7947 - loss: 0.5002
Epoch 4: val_loss improved from 0.50867 to 0.44554, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.7919 - loss: 0.5004 - val_accuracy: 0.8235 - val_loss: 0.4455
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7860 - loss: 0.4799
Epoch 5: val_loss improved from 0.44554 to 0.41329, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.7957 - loss: 0.4304 - val_accuracy: 0.8431 - val_loss: 0.3817
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8163 - loss: 0.4196
Epoch 7: val_loss improved from 0.38168 to 0.35999, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.8213 - loss: 0.4165 - val_accuracy: 0.8627 - val_loss: 0.3600
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8739 - loss: 0.3500
Epoch 8: val_loss improved from 0.35999 to 0.34341, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.8736 - loss: 0.3500 - val_accuracy: 0.8627 - val_loss: 0.3434
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8690 - loss: 0.3728
Epoch 9: val_loss improved from 0.34341 to 0.32863, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.8718 - loss: 0.3698 - val_accuracy: 0.8824 - val_loss: 0.3286
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9016 - loss: 0.2954
Epoch 10: val_loss improved from 0.32863 to 0.31640, saving model to /content/drive/MyDrive/best_brain_tumor_model.h5


7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.8978 - loss: 0.2996 - val_accuracy: 0.9020 - val_loss: 0.3164


In [8]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/best_brain_tumor_model.h5")

In [9]:
from tensorflow.keras.optimizers import Adam
model.layers[2].trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_cb = ModelCheckpoint(
    "/content/drive/MyDrive/fine_tuned_model.keras",
    save_best_only=True,
    monitor="val_loss",
    mode="min",
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [12]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[checkpoint_cb, early_stop]
)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8613 - loss: 0.3700
Epoch 1: val_loss improved from 0.37590 to 0.37555, saving model to /content/drive/MyDrive/fine_tuned_model.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.8595 - loss: 0.3714 - val_accuracy: 0.8627 - val_loss: 0.3755
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8696 - loss: 0.3905
Epoch 2: val_loss improved from 0.37555 to 0.37518, saving model to /content/drive/MyDrive/fine_tuned_model.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.8674 - loss: 0.3906 - val_accuracy: 0.8627 - val_loss: 0.3752
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8473 - loss: 0.3967
Epoch 3: val_loss improved from 0.37518 to 0.37488, saving model to /content/drive/MyDrive/fine_tuned_model.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.8459 - loss: 0.3968 - val_accuracy: 0.8627 - val_loss: 0.3749
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8411 - loss: 0.